In [29]:
import pandas as pd
import numpy as np
import glob
import time
import os
import pymarc
import marcx
import codecs
import string
os.chdir('/Volumes/jwa_drive1/git/pardee')

In [2]:
df_contents = pd.read_csv('contents.csv',sep=',')
df_contents = df_contents.dropna()
df_sources = pd.read_csv('source.csv',sep=',')
df_surveys = pd.read_csv('survey.csv', sep=',',dtype={'naics':str})
df_crosswalk = pd.read_excel('SIC_NASIC_Crosswalk.xlsx')
df_crosswalk.rename(columns={'NAICS Industry\xa0':'NAICS Industry'}, inplace=True)
df_crosswalk['NAICS'] = df_crosswalk['NAICS'].astype(str)
df_crosswalk['SIC'] = df_crosswalk['SIC'].astype(str)
df_surveys['naics'] = df_surveys['naics'].astype(str)
df_surveys['sic'] = df_surveys['sic'].astype(str)
header = '<?xml version="1.0" encoding="UTF-8" ?><collection xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/MARC21/slim http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd">'
footer = '</collection>'
print(df_surveys.columns)
print(df_contents.columns)
print(df_sources.columns)
#df_sources.set_index('c_so_id')


Index(['sur_id', 'title', 'sic', 'keyword', 'naics'], dtype='object')
Index(['c_id', 'c_sur_id', 'c_so_id', 'c_url'], dtype='object')
Index(['so_id', 's_title', 'location', 'type', 'u_type', 's_url', 'g_url',
       'ezpx'],
      dtype='object')


In [3]:
def norm_str(s):
    exclude = set(string.punctuation)
    exclude.add(' ')
    if not type(s) == str:
        s = str(s)
    s = ''.join(ch for ch in s if ch not in exclude)
    #s=s.replace(' ','')
    return(s.lower())

def get_sic(s,df):
    #print(type(df.ntitle))
    #print(type(s.ntitle))
    mask = df.ntitle == s.ntitle
    d = df[mask]
    #print(type(d))
    return(d)
    

In [4]:
out = codecs.open('PardeeIS.xml','w', 'utf-8')
out.write(header)
counter = 0
no_856 = 0
proxy_list = ['galegroup.com/','lexisnexis.com/','www.mergentonline.com/','standardandpoors.com/','valueline.com/','proquest.com/','ibisworld.com/','ebscohost.com/']
for index,row in df_surveys.iterrows():
    ## default leader and 008 values
    leader_value = '#####naa#a22#####3##4500'
    _008_value = '######s2015####xxu####eote###000#0#eng#d'
    ## grab the values for sic, naics, keywords from the dataframe
    sic = row['sic']
    naics = row['naics']
    keywords = row['keyword']
    title = row['title']
    ## create empty lists for sics, urls
    sics = []
    urls = []
    ## if no value for sic, lookup in df_crosswalk dataframe
    if sic == 'nan':
        mask = df_crosswalk['NAICS'] == naics
        for index,r in df_crosswalk[mask].iterrows():
            sics.append(r['SIC'])
    else:
        sics.append(sic)
    try:
        sic = sics[0]
    except:
        sic = ''
    ## check for multiple values in the naics field. If so, split them into a list
    naics = naics.replace(' ','')
    if naics.find(',') > 0:
        naics = naics.split(',')
    else:
        naics = [naics]
    ##
    ## get the url
    ##
    mask = df_contents['c_sur_id'] == row['sur_id']
    for index,r in df_contents[mask].iterrows():
        smask = df_sources['so_id'] == r['c_so_id']
        for s_i,s_r in df_sources[smask].iterrows():
            s_title = s_r['s_title']
            s_location = s_r['location']
        urls.append([r['c_url'],s_title,s_location])
    ## create a list of naics
    mask = df_crosswalk['SIC'] == row['sic']
    naics_rows = df_crosswalk[mask]
    ## create a marc record and add fields
    record = marcx.FatRecord()
    record.leader = leader_value
    record.add('008',data = _008_value)
    for n in sics:
        if n != 'nan':
            record.add('024', a=str(n),_2='SICS',indicators=['7',' '])
    for n in naics:
        if n != 'nan':
            record.add('024', a=str(n),_2='NAICS',indicators=['7',' '])
    record.add('041',a='eng')
    record.add('245',a = str(title),indicators=['0','0'])
    ## lookup the NAICS Industry description and add it to a note field
    for index,row in df_crosswalk[mask].iterrows():
        try:
            record.add('500',a = row['NAICS Industry'])
        except:
            pass
    ## add the keywords to a 650 field. Is this the best field?
    record.add('650',a = str(keywords),indicators=['0','4'])
    ## iterate through the urls and add to an 856
    ## need to add a subfield z and subfield 3 with the title and source from the df_sources dataframe
    ##
    record.add('902',a='PardeeIndustrySurvey')
    for n in urls:
        record.remove('856')
        url = str(n[0])
        s_tit = str(n[1])
        s_loc = str(n[2])
        s_link_text = ''
        if s_tit == 'Encyclopedia of American Industries' and len(sic) > 0:
            s_link_text = 'Search SIC [' + sic +'] in Quick Search Box'
        for px in proxy_list:
            if url.find(px) > 0:
                url = url.replace(px,px[:-1]+ '.ezproxy.bu.edu/')
        if s_tit.find('Encyclopedia') == 0:
            record.add('856',u=url,z=s_tit,n=s_loc,y=s_link_text,indicators=['4','0'])
        else:
            record.add('856',u=url,z=s_tit,n=s_loc,indicators=['4','0'])

        if record.has('856'):
            out.write(pymarc.record_to_xml(record ).decode("utf-8"))
            counter += 1
            try:
                print('Record: ',str(counter))
                #print(record)
            except Exception as e:
                print(e)
        else:
            no_856 += 1
            print()
            print("no 856",str(no_856))
            print(record)
out.write(footer)

Record:  1
Record:  2
Record:  3
Record:  4
Record:  5
Record:  6
Record:  7
Record:  8
Record:  9
Record:  10
Record:  11
Record:  12
Record:  13
Record:  14
Record:  15
Record:  16
Record:  17
Record:  18
Record:  19
Record:  20
Record:  21
Record:  22
Record:  23
Record:  24
Record:  25
Record:  26
Record:  27
Record:  28
Record:  29
Record:  30
Record:  31
Record:  32
Record:  33
Record:  34
Record:  35
Record:  36
Record:  37
Record:  38
Record:  39
Record:  40
Record:  41
Record:  42
Record:  43
Record:  44
Record:  45
Record:  46
Record:  47
Record:  48
Record:  49
Record:  50
Record:  51
Record:  52
Record:  53
Record:  54
Record:  55
Record:  56
Record:  57
Record:  58
Record:  59
Record:  60
Record:  61
Record:  62
Record:  63
Record:  64
Record:  65
Record:  66
Record:  67
Record:  68
Record:  69
Record:  70
Record:  71
Record:  72
Record:  73
Record:  74
Record:  75
Record:  76
Record:  77
Record:  78
Record:  79
Record:  80
Record:  81
Record:  82
Record:  83
Record:  84
R

In [5]:
ser = df_surveys['title']
ser.name = 'ntitle'
df_surveys = pd.concat([df_surveys,ser],axis=1)
for index,row in df_surveys.iterrows():
    ntitle = norm_str(row['title'])
    #print(ntitle)
    df_surveys.loc[df_surveys['title'] == row['title'], 'ntitle'] = ntitle

#df_crosswalk.head()

ser = df_crosswalk['SIC Industry']
ser.name = 'ntitle'
df_crosswalk = pd.concat([df_crosswalk,ser],axis=1)
for index,row in df_crosswalk.iterrows():
    ntitle = norm_str(row['SIC Industry'])
    #print(ntitle)
    df_crosswalk.loc[df_crosswalk['SIC Industry'] == row['SIC Industry'], 'ntitle'] = ntitle
#df_crosswalk.head()

In [6]:
#df_surveys.head()
mask = ((df_surveys['sic'] == 'nan') & (df_surveys['naics'] == 'nan'))
df_surveys[mask]
nosic_nonaics = df_surveys[mask]
print(nosic_nonaics.shape)
nosic_nonaics.head()
matches = 0
for index,row in nosic_nonaics.iterrows():
    #nosic_nonaics.loc[nosic_nonaics['ntitle'] == row['ntitle'],'sic'] = get_sic(row,df_crosswalk)
    df_ret = get_sic(row,df_crosswalk)
    if df_ret.shape[0] > 0:
        sic = str(df_ret['SIC'])
        sic= sic[:sic.index('\n')]
        sic = sic[sic.rindex(' '):].strip()
        print(row.title,sic)
        nosic_nonaics.loc[nosic_nonaics['ntitle'] == row['ntitle'],'sic'] = sic


(1821, 6)
Books 2730
Footwear 5139
Book Publishing 2731
Car Washes 7542
Bowling Centers 7933
Bookstores 5942
Boat Dealers 5551
Engineering Services 8711
Department Stores 5311
Footwear 5139
Life Insurance 6311
Natural Gas: Distribution 4924
Colleges and Universities 8221
Florists 5992
Furniture Stores 5712
Title Insurance 6361
Highway and Street Construction 1611
Home Healthcare Services 8082
Kidney Dialysis Centers 8092
Legal Services 8111
Marinas 4493
Musical Instrument Stores 5736
Petroleum Refining 2910
Recreational Vehicle Dealers 5560
Shoe Stores 5660
Testing Laboratories 8734
Used Merchandise Stores 5932
Veterinary Services 740
Medical Equipment Rental 7352
Dental Laboratories 8072


/Users/jammerman/anaconda/envs/py3k/lib/python3.3/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
mask = ((df_surveys['sic'] == 'nan') & (df_surveys['naics'] == 'nan'))
df_surveys[mask]
nosic_nonaics = df_surveys[mask]
print(nosic_nonaics.shape)
nosic_nonaics.head()

(1821, 6)


,sur_id,title,sic,keyword,naics,ntitle
16,17,Adult Entertainment,nan,pornographic porn,nan,adultentertainment
17,18,Adult Obesity,nan,overweight over weight obese healthcare health...,nan,adultobesity
18,19,Advertising,nan,ads direct-mail marketing agencies services,nan,advertising
21,24,Advertising and Marketing,nan,advertising,nan,advertisingandmarketing
26,32,Aerospace/Defense,nan,defense aircraft military weapons missiles air...,nan,aerospacedefense


In [10]:
for index,row in no.iterrows():
    ntitle = norm_str(row['title'])
    #print(ntitle)
    no.loc[no['title'] == row['title'], 'ntitle'] = ntitle

no.head(15)


NameError: name 'no' is not defined

In [11]:
no.loc[3793,'ntitle'] =row['ntitle']


NameError: name 'no' is not defined

In [ ]:
df_crosswalk.head()  

In [ ]:
sics

In [12]:
df_surveys

,sur_id,title,sic,keyword,naics,ntitle
0,1,Abrasive Products,3291,fabricated metals grinding industrial equipme...,332999,abrasiveproducts
1,2,AC Drives,nan,electronics industrial equipment electric pow...,221122,acdrives
2,3,AC Drives: Global,nan,electronics industrial equipment electric pow...,221122,acdrivesglobal
3,4,Accident and Health Insurance,6321,Health Medical Insurance Carriers carrier Emp...,"524114, 525190, 524130",accidentandhealthinsurance
4,5,Accountancy,nan,accounting tax services accountants designing ...,541211,accountancy
5,6,Accountancy: Global,nan,accounting tax services accountants designing ...,541211,accountancyglobal
6,7,"Accounting, Auditing, and Bookkeeping Services",8721,accountants cpas payroll service accounting bo...,"541211, 541214, 541219",accountingauditingandbookkeepingservices
7,8,Adhesives and Sealants,2891,manufacturing industrial household glues caul...,325520,adhesivesandsealants
8,9,Adjustment and Collection Services,7322,collection agencies repossession services clai...,"561440, 561491",adjustmentandcollectionservices
9,10,Administration of Educational Programs,9411,educational scholarships education central coo...,923110,administrationofeducationalprograms


In [76]:
records = pymarc.parse_xml_to_array(codecs.open('PardeeIS.xml','rb','utf-8'))

In [77]:
url_dict = {}
counter = 0
clientsibisworld = [] 
wwwibisworld = [] 
umi = []
lexisnexis = []
mintel = [] 
galegroup = []
ebscohost = [] 
mergentonline = [] 
doi = []
for rec in records:
    rec = marcx.FatRecord.from_record(rec)

    domain = rec['856']['u'][:rec['856']['u'].rindex('/')]
#    domain = domain[domain.index('.')+1:domain.index('.',domain.index('.')+1)]
    domain = domain[domain.index(':')+3:domain.index('.',domain.index('.')+1)] 
    #print(domain)
    if domain == 'www.ibisworld':
        rec.remove('856')
        rec.add('856', u='http://clients1.ibisworld.com/reports/us/industry/home.aspx',z='IBISWorld Industry Reports',n='IBISWorld',indicators='40')
        clientsibisworld.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'clients1.ibisworld':
        rec.remove('856')
        rec.add('856', u='http://clients1.ibisworld.com/reports/us/industry/home.aspx',z='IBISWorld Industry Reports',n='IBISWorld',indicators='40')
        clientsibisworld.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'clients.ibisworld':
        rec.remove('856')
        rec.add('856', u='http://clients1.ibisworld.com/reports/us/industry/home.aspx',z='IBISWorld Industry Reports',n='IBISWorld',indicators='40')
        clientsibisworld.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'www.lexisnexis':
        url = rec['856']['u'].replace('.ezproxy.bu.edu','')
        rec.remove('856')
        rec.add('856', u=url,z='Hoovers Industry Snapshots',n='Lexis Nexis',indicators='40')
        lexisnexis.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'proquest.umi':
        rec.remove('856')
        rec.add('856', u='http://search.proquest.com/browseterms/firstResearch_business?accountid=9676',z='First Research Reports',n='ABI-Inform',indicators='40')
        umi.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'academic.mintel':
        mintel.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'find.galegroup':
        url = rec['856']['u'].replace('.ezproxy.bu.edu','')
        _z = rec['856']['z']
        _n = rec['856']['n']
        rec.remove('856')
        rec.add('856', u=url,z=_z,n=_z,indicators='40')
        galegroup.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'search.ebscohost':
        url = rec['856']['u'].replace('.ezproxy.bu.edu','')
        _z = rec['856']['z']
        _n = rec['856']['n']
        rec.remove('856')
        rec.add('856', u=url,z=_z,n=_z,indicators='40')
        ebscohost.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'www.mergentonline':
        url = rec['856']['u'].replace('.ezproxy.bu.edu','')
        _z = rec['856']['z']
        _n = rec['856']['n']
        rec.remove('856')
        rec.add('856', u=url,z=_z,n=_z,indicators='40')
        mergentonline.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
    if domain == 'dx.doi':
        url = rec['856']['u'].replace('.ezproxy.bu.edu','')
        _z = rec['856']['z']
        _n = rec['856']['n']
        rec.remove('856')
        rec.add('856', u=url,z=_z,n=_z,indicators='40')
        doi.append(rec)
        if rec['856']['u'][:rec['856']['u'].rindex('/')] in url_dict:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] += 1
        else:
            url_dict[rec['856']['u'][:rec['856']['u'].rindex('/')]] = 1
                    
for k,v in url_dict.items():
    print(str(counter),k,str(v))
    counter += 1

0 http://dx.doi.org/10.1787 8
1 http://academic.mintel.com 562
2 http://www.lexisnexis.com/us/lnacademic/api/version1 127
3 http://find.galegroup.com/gvrl 996
4 http://clients1.ibisworld.com/reports/us/industry 1213
5 http://www.lexisnexis.com/us/lnacademic/search 1
6 http://www.mergentonline.com 25
7 http://search.proquest.com/browseterms 326
8 http://search.ebscohost.com 409


In [78]:
groups = [(clientsibisworld,'ibis.xml'),(umi,'umi.xml'),
          (lexisnexis,'lexisnexis.xml'),(mintel,'mintel.xml'),
          (galegroup,'galegroup.xml'),(ebscohost,'ebscohost.xml'),
          (mergentonline,'mergentonline.xml'),(doi,'doi.xml'),]
for group in groups:
    print()
    file_name = group[1]
    out = codecs.open(file_name,'w','utf-8')
    print(file_name)
    records = group[0]  
    out.write(header)
    for rec in records:
        out.write(pymarc.record_to_xml(rec ).decode("utf-8"))
        #print(rec)
    out.write(footer)
    out.close()


ibis.xml

umi.xml

lexisnexis.xml

mintel.xml

galegroup.xml

ebscohost.xml

mergentonline.xml

doi.xml
